# Consigna NX - Challenge.

Cada una de las filas del archivo (indexada por SK_ID_CURR) corresponde a una persona distinta que nos pidió un préstamo, y la columna "TARGET" dice qué ocurrió con ese pedido en particular (es decir, tiene un 1 si su pago se atrasó , es decir, que entraron en default, y 0 si no lo hizo). La idea es que puedas hacer un proceso que permita predecir los clientes futuros que van a entrar en default sobre el préstamo.
El archivo "HomeCredit_columns_description - HomeCredit_columns_description.parquet" tiene la información disponible sobre cada una de las columnas de los otros archivos adjuntos. 

Para poder orientarte nuestra evaluación, vamos a hacer foco en el EDA y en el modelo predictivo, sin entrar en profundidad si la métrica esta tuneada o no.

# Modelo de clasificación

In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
import seaborn as sns

In [3]:
df_app = pd.read_csv('/Users/santiagogimenez/Documents/GitHub/nx-challenge/outputs/preprocessed_application.csv')
df_ins = pd.read_csv('/Users/santiagogimenez/Documents/GitHub/nx-challenge/outputs/preprocessed_instalment.csv')
df_cre = pd.read_csv('/Users/santiagogimenez/Documents/GitHub/nx-challenge/outputs/preprocessed_credit.csv')

In [4]:
df_app.drop(columns=df_app.columns[0], inplace=True)
df_app

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,0,1,202500.0,406597.5,24700.5,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,0,0,270000.0,1293502.5,35698.5,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,1,1,67500.0,135000.0,6750.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,0,1,135000.0,312682.5,29686.5,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,0,1,121500.0,513000.0,21865.5,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307504,456251,0,Cash loans,M,0,0,157500.0,254700.0,27558.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307505,456252,0,Cash loans,F,0,1,72000.0,269550.0,12001.5,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307506,456253,0,Cash loans,F,0,1,153000.0,677664.0,29979.0,Unaccompanied,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307507,456254,1,Cash loans,F,0,1,171000.0,370107.0,20205.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_ins.drop(columns=df_ins.columns[0], inplace=True)
df_ins

,SK_ID_CURR,AMT_PAYMENT_INSTALMENT,DAYS_ENTRY_INSTALMENT
0,100001,1.000000,1.004767
1,100002,1.000000,1.131260
2,100003,1.000000,1.007302
3,100004,1.000000,1.010037
4,100005,1.000000,1.039191
...,...,...,...
339582,456251,1.000000,1.357098
339583,456252,1.000000,1.001208
339584,456253,0.928571,1.007221
339585,456254,1.000000,1.293806


In [6]:
df_cre

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_TOTAL_DRAWINGS,AMT_PAYMENT_TOTAL_CURRENT,CNT_TOTAL_DRAWINGS,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
0,100006,-3.5,0.000000,270000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
1,100011,-38.5,54482.111149,164189.189189,4864.864865,4520.067568,0.108108,25.767123,0.000000,0.000000
2,100013,-48.5,18159.919219,131718.750000,12700.000000,6817.172344,0.511111,18.719101,0.010417,0.010417
3,100021,-10.0,0.000000,675000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
4,100023,-7.5,0.000000,135000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
103553,456244,-21.0,131834.730732,296341.463415,53681.013659,32720.544878,2.731707,13.600000,0.000000,0.000000
103554,456246,-5.5,13136.731875,135000.000000,30398.512500,15554.340000,5.000000,3.500000,0.000000,0.000000
103555,456247,-49.0,23216.396211,144000.000000,4299.012947,4115.878105,0.294737,26.494737,0.031579,0.021053
103556,456248,-13.0,0.000000,900000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000


In [7]:
# seteo el SK_ID_CURR como indice para poder unir los datasets

df_app.set_index(['SK_ID_CURR'], inplace=True)
df_ins.set_index(['SK_ID_CURR'],  inplace=True)
df_cre.set_index(['SK_ID_CURR'],  inplace=True)

In [8]:
df = pd.concat([df_app,df_ins,df_cre], axis=1, join='inner')
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,DAYS_ENTRY_INSTALMENT,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_TOTAL_DRAWINGS,AMT_PAYMENT_TOTAL_CURRENT,CNT_TOTAL_DRAWINGS,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,0,Cash loans,F,0,1,135000.0,312682.5,29686.5,Unaccompanied,Working,...,1.064315,-3.5,0.000000,270000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
100011,0,Cash loans,F,0,1,112500.0,1019610.0,33826.5,Children,Pensioner,...,1.001933,-38.5,54482.111149,164189.189189,4864.864865,4520.067568,0.108108,25.767123,0.000000,0.000000
100021,0,Revolving loans,F,0,1,81000.0,270000.0,13500.0,Unaccompanied,Working,...,1.175151,-10.0,0.000000,675000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
100023,0,Cash loans,F,0,1,90000.0,544491.0,17563.5,Unaccompanied,State servant,...,1.020944,-7.5,0.000000,135000.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
100036,0,Cash loans,F,0,1,112500.0,512064.0,25033.5,Family,Working,...,1.096231,-7.5,0.000000,78750.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456242,0,Cash loans,M,1,1,198000.0,1312110.0,52168.5,Unaccompanied,Commercial associate,...,1.144239,-4.5,148232.328750,202500.000000,79699.567500,19580.158125,7.750000,3.500000,0.000000,0.000000
456244,0,Cash loans,F,0,1,261000.0,1303812.0,35982.0,Unaccompanied,Working,...,1.014726,-21.0,131834.730732,296341.463415,53681.013659,32720.544878,2.731707,13.600000,0.000000,0.000000
456246,0,Cash loans,F,0,1,94500.0,225000.0,10620.0,Unaccompanied,Working,...,1.018575,-5.5,13136.731875,135000.000000,30398.512500,15554.340000,5.000000,3.500000,0.000000,0.000000


In [9]:
df['TARGET'].mean() * 100

8.682057849618682

In [10]:
#  cantidad de valores nulos:

pd.DataFrame({'cant' : df.isna().sum(),
                'perc' : df.isna().sum()/len(df)*100}).sort_values(['cant']).tail(20)

,cant,perc
FLAG_PHONE,0,0.000000
FLAG_CONT_MOBILE,0,0.000000
FLAG_WORK_PHONE,0,0.000000
CNT_FAM_MEMBERS,0,0.000000
DAYS_ENTRY_INSTALMENT,2,0.002308
AMT_PAYMENT_INSTALMENT,7,0.008076
AMT_ANNUITY,8,0.009230
EXT_SOURCE_2,22,0.025383
OBS_60_CNT_SOCIAL_CIRCLE,35,0.040382
DEF_60_CNT_SOCIAL_CIRCLE,35,0.040382


voy a eliminar las ultimas dos columnas de mi dataset por tener mas del 20% de datos faltantes.

In [11]:
df_nan = df.drop(columns=['AMT_TOTAL_DRAWINGS','CNT_TOTAL_DRAWINGS'])
df_nan

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_PAYMENT_INSTALMENT,DAYS_ENTRY_INSTALMENT,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_PAYMENT_TOTAL_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,0,Cash loans,F,0,1,135000.0,312682.5,29686.5,Unaccompanied,Working,...,NaN,1.000000,1.064315,-3.5,0.000000,270000.000000,0.000000,0.000000,0.000000,0.000000
100011,0,Cash loans,F,0,1,112500.0,1019610.0,33826.5,Children,Pensioner,...,1.0,0.907692,1.001933,-38.5,54482.111149,164189.189189,4520.067568,25.767123,0.000000,0.000000
100021,0,Revolving loans,F,0,1,81000.0,270000.0,13500.0,Unaccompanied,Working,...,NaN,1.000000,1.175151,-10.0,0.000000,675000.000000,0.000000,0.000000,0.000000,0.000000
100023,0,Cash loans,F,0,1,90000.0,544491.0,17563.5,Unaccompanied,State servant,...,2.0,1.000000,1.020944,-7.5,0.000000,135000.000000,0.000000,0.000000,0.000000,0.000000
100036,0,Cash loans,F,0,1,112500.0,512064.0,25033.5,Family,Working,...,5.0,1.000000,1.096231,-7.5,0.000000,78750.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456242,0,Cash loans,M,1,1,198000.0,1312110.0,52168.5,Unaccompanied,Commercial associate,...,2.0,1.000000,1.144239,-4.5,148232.328750,202500.000000,19580.158125,3.500000,0.000000,0.000000
456244,0,Cash loans,F,0,1,261000.0,1303812.0,35982.0,Unaccompanied,Working,...,5.0,0.939850,1.014726,-21.0,131834.730732,296341.463415,32720.544878,13.600000,0.000000,0.000000
456246,0,Cash loans,F,0,1,94500.0,225000.0,10620.0,Unaccompanied,Working,...,1.0,1.000000,1.018575,-5.5,13136.731875,135000.000000,15554.340000,3.500000,0.000000,0.000000


In [12]:
df_nan.drop(df_nan[df_nan['NAME_INCOME_TYPE'] == 'Maternity leave'].index[0], axis=0, inplace=True)

In [13]:
X = df_nan.iloc[:,1:]
y = df_nan.iloc[:,0]

In [14]:
numerical_columns = [x for x in X.columns if X[x].dtype != 'object' and X[x].dtype != 'bool']
cat_columns = [x for x in X.columns if x not in numerical_columns]
int_columns = [x for x in numerical_columns if X[x].dtypes != 'float64']
float_columns = [x for x in numerical_columns if x not in int_columns]

print('cat:',len(cat_columns),'int:', len(int_columns),'float:', len(float_columns))

cat: 9 int: 38 float: 26


In [15]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 , random_state=42)

In [16]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(60670, 73) (26002, 73) (60670,) (26002,)


In [17]:
#Ahora tengo que rellenar las columnas con datos faltantes.

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

cat_mf = cat_columns
cat_med = float_columns

pipe_ohe = Pipeline([
    ('ohe', OneHotEncoder(drop='first')),
    ('ohe_mf', SimpleImputer(strategy='most_frequent')),
])

pipe_mf = Pipeline([
                    ('mf_imputer',SimpleImputer(strategy='most_frequent')),
                    ])

pipe_med = Pipeline([
                     ('med_imputer',SimpleImputer(strategy='mean')),
                     ('scaling',StandardScaler())
                     ])

pipe_columns = ColumnTransformer([
        ("mf", pipe_mf, int_columns),
        ('med',pipe_med, float_columns),
        ("ohe", pipe_ohe, cat_columns)
    ])

X_train_processed = pipe_columns.fit_transform(X_train)
X_train_labels = pipe_columns.get_feature_names_out(X_train.columns)

X_test_processed = pipe_columns.fit_transform(X_test)
X_test_labels = pipe_columns.get_feature_names_out(X_test.columns)




In [18]:
df_train = pd.DataFrame(X_train_processed, columns=X_train_labels)
df_test = pd.DataFrame(X_test_processed, columns=X_test_labels)

In [19]:
# este dato me esta molestando.

(df_train.columns).symmetric_difference(df_test.columns)

Index([], dtype='object')

In [20]:
# hago un oversampling de la clase minoritaria para mi dataset de entrenamiento

from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42)

df_train_res, y_train_res = sm.fit_resample(df_train, y_train)


In [21]:
print(df_train_res.shape, df_test.shape)

(110810, 108) (26002, 108)


In [22]:
y_train_res.mean()

0.5

In [23]:
# solamente tengo un solo valor de esta columna, por lo que la tiro para tener las mismas dimensiones en ambos datasets de train y test.
#df_train.drop(columns=['ohe__NAME_INCOME_TYPE_Maternity leave'], inplace=True)

In [24]:
# me inicio un modelo de clasificación de XGBoost
import xgboost as xgb

xgb_model = xgb.XGBClassifier()

# Me inicio un gridsearch para tunear los hyperparametros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

    # grilla de parametros para XGBoost
params = {
        'min_child_weight': [1, 10],
        'gamma': [0.5, 1.5, 5],
        'subsample': [0.6, 1.0],
        'colsample_bytree': [0.6, 1.0],
        'max_depth': [2, 3]
        }

# Defino el modelo con la crossvalidación por GridSearch

model = RandomizedSearchCV(xgb_model, params, n_iter=10, cv=5)

#Entreno el modelo

model.fit(df_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_cat_to_onehot=None,
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   param_distributions={'colsample_bytree': [0.6, 1.0],
                                        'gamma': [0.5, 1.5, 5],
                                        'max_depth': [2, 3],
                                        'min_child_weight': [1, 10],
                                        'subsample': [0.6, 1.0]})

In [25]:

# Imprimo en pantalla los mejores parametros para utilizar

print("Mejores parametros segun GRID: "+str(model.best_params_))
print("Mejor Score segun GRID: "+str(model.best_score_)+'\n')

Mejores parametros segun GRID: {'subsample': 1.0, 'min_child_weight': 10, 'max_depth': 2, 'gamma': 0.5, 'colsample_bytree': 1.0}
Mejor Score segun GRID: 0.9137135322235043



In [26]:
# Me genero predicciones para testear mi modelo
y_pred = model.predict(df_test)

In [27]:
from sklearn.metrics import accuracy_score , recall_score , f1_score , precision_score , classification_report , confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

pd.DataFrame({
    'accuracy' : [accuracy],
    'recall' : [recall],
    'f1' : [f1],
    'precision' : [precision]
}, index=['metric_values'])



,accuracy,recall,f1,precision
metric_values,0.912545,0.034529,0.064198,0.45614
